In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
%matplotlib inline

In [ ]:
def dynasty(t, y, R, E, B=0.17, D=0.42, G=0.09, H=0.1, Q=0.4):
    return np.array([
        y[0] * (1 - y[0] - y[1] / (B + y[0]) - H * y[2]),
        Q * y[1] * (E * y[0] / (B + y[0]) - 1 - y[2] / (D + y[1])),
        R * (y[0] * y[1] / (B + y[0]) - G * y[2])
    ])

def jacobian(t, y, R, E, B=0.17, D=0.42, G=0.09, H=0.1, Q=0.4):
    deny0 = 1 / (B + y[0]);
    deny1 = 1 / (D + y[1]);
    deny0square = deny0 ** 2;
    deny1square = deny1 ** 2;
    return np.array([
        [1 - 2 * y[0] - B * y[1] * deny0square - H * y[2], -y[0] * deny0, -H * y[0]],
        [Q * E * B * y[1] * deny0square, Q * E * y[0] * deny0 - Q * D * y[2] * deny1square - Q, -Q * y[1] * deny1],
        [R * B * y[1] * deny0square, R * y[0] * deny0, -R * G]
    ])

def y0min(t, y, B=0.17, H=0.1):
    return y[0] * (1 - y[0] - y[1] / (B + y[0]) - H * y[2])
y0min.direction = 1

def y1max(t, y, E, B=0.17, D=0.42, H=0.1, Q=0.4):
    return Q * y[1] * (E * y[0] / (B + y[0]) - 1 - y[2] / (D + y[1]))
y1max.direction = 1

In [ ]:
reltol = 1e-12
abstol = 1e-10 + np.zeros(3)
tend = 20000.
R = 0.1
E = 2.3
y0 = np.array([0.8,0.1,0.1])
sol = solve_ivp(lambda t,y: dynasty(t, y, R, E),
                [0,tend], y0, method='BDF',
                jac=lambda t,y: jacobian(t, y, R, E),
                events=lambda t,y: y1max(t, y, E), dense_output=True,
                rtol=reltol, atol=abstol)
t = sol['t']
y = sol['y']

In [ ]:
sol['y'][:,-1]

In [ ]:
t_ev = sol['t_events'][0]
y_ev = sol['sol'](t_ev)

In [ ]:
ttran = 19000

fig,(ax1,ax2) = plt.subplots(1, 2, figsize=(12,5))

idx = t > ttran
col = 'kgm'
for i in range(3):
    ax1.plot(t[idx], y[i][idx], col[i], lw=1, label=r'$\mathrm{x}_%d$' % (i+1))
ax2.plot(y[2][t > ttran/5], y[1][t > ttran/5], 'k', lw=1)
idx = t_ev > ttran
ax1.plot(t_ev[idx], y_ev[1][idx], 'ro', markerfacecolor='w', markersize=5, markeredgewidth=1)
ax1.legend(loc='best')
ax1.set_xlabel('Time')
ax1.set_ylim([0, 1])

ax2.set_xlabel(r'$\mathrm{x}_3$')
ax2.set_ylabel(r'$\mathrm{x}_2$')
ax2.set_xlim([0, 1])
ax2.set_ylim([0, 1])

fig.tight_layout()